# transformers推理

## 创建模型
根据本地模型地址，加载分词器和语言模型

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 设定使用的设备为CUDA，如果CUDA不可用，将回退到CPU
device = "cuda"

# 指定预训练模型的路径
mode_name_or_path = '/root/autodl-tmp/ZhipuAI/glm-4-9b-chat'

# 从指定路径加载预训练的分词器
tokenizer = AutoTokenizer.from_pretrained(mode_name_or_path, trust_remote_code=True)


# 从指定路径加载预训练的模型
# torch_dtype指定数据类型，low_cpu_mem_usage优化CPU内存使用，trust_remote_code允许加载远程代码
model = AutoModelForCausalLM.from_pretrained(
    mode_name_or_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to(device).eval()  # 将模型设置为评估模式


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

## 定义提示消息
定义提示消息，对输入消息进行分词处理

In [2]:
# 定义用户输入的查询文本
query = "请介绍一下AI大模型"

# 使用分词器的apply_chat_template方法来准备输入数据
# 这个方法会根据聊天模板将用户输入格式化为模型可接受的格式
# add_generation_prompt添加生成提示，tokenize进行分词，return_tensors指定返回PyTorch张量
# return_dict指定返回字典格式，方便后续处理
inputs = tokenizer.apply_chat_template(
    [{"role": "user", "content": query}],
    add_generation_prompt=True,
    tokenize=True,
    return_tensors="pt",
    return_dict=True
)

# 将输入数据移动到指定的设备上
inputs = inputs.to(device)

## 模型文本生成
利用模型，根据输入的input，生成输出的响应output

In [3]:
# 定义生成文本时的参数
gen_kwargs = {
    "max_length": 2500,  # 设置生成文本的最大长度
    "do_sample": True,  # 是否从可能的下一个词中随机选择
    "top_k": 1  # 从概率最高的k个词中选择
}

# 使用torch.no_grad()上下文管理器来禁用梯度计算，这在推理时可以减少内存使用
with torch.no_grad():
    # 使用模型的generate方法生成文本
    outputs = model.generate(**inputs, **gen_kwargs)
    
    # 截取生成的文本，去除开头的提示部分
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    
    # 使用分词器的decode方法将生成的词ID解码回文本，并打印出来
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


AI大模型，即人工智能大型模型，是指那些具有海量参数和强大计算能力的深度学习模型。这些模型通常用于处理复杂的自然语言处理（NLP）、计算机视觉、语音识别等任务。以下是关于AI大模型的一些详细介绍：

1. **定义**：
   - AI大模型是指具有数十亿甚至数千亿参数的深度学习模型。
   - 这些模型通常采用大规模数据集进行训练，以实现高水平的性能。

2. **应用领域**：
   - **自然语言处理**：如机器翻译、文本摘要、问答系统等。
   - **计算机视觉**：如图像识别、目标检测、图像生成等。
   - **语音识别**：如语音转文字、语音合成等。
   - **推荐系统**：如个性化推荐、商品推荐等。

3. **特点**：
   - **强大的计算能力**：AI大模型需要大量的计算资源进行训练和推理。
   - **海量参数**：这些模型通常具有数十亿甚至数千亿个参数，能够捕捉到数据中的复杂模式。
   - **自学习能力**：AI大模型能够从大量数据中自动学习，无需人工干预。
   - **泛化能力**：这些模型在训练数据之外的未知数据上也能表现出良好的性能。

4. **挑战**：
   - **计算资源**：训练和推理AI大模型需要大量的计算资源，如GPU、TPU等。
   - **数据隐私**：AI大模型通常需要大量数据来训练，这可能会引发数据隐私问题。
   - **模型可解释性**：AI大模型的决策过程往往难以解释，这可能会影响其在实际应用中的可信度。

5. **代表性模型**：
   - **Transformer**：一种基于自注意力机制的深度学习模型，广泛应用于NLP任务。
   - **BERT**（Bidirectional Encoder Representations from Transformers）：一种基于Transformer的预训练语言模型，在NLP任务中取得了显著成果。
   - **GPT**（Generative Pre-trained Transformer）：一种基于Transformer的生成式语言模型，能够生成高质量的文本。

总之，AI大模型是人工智能领域的一个重要研究方向，具有广泛的应用前景。随着技术的不断发展，AI大模型将在更多领域发挥重要作用。
